1. setup fastai: https://github.com/corykendrick/fastai_in_colab/blob/master/Using_Google_Colab_for_Fastai.ipynb

2. setup kaggle api: https://medium.com/@yvettewu.dw/tutorial-kaggle-api-google-colaboratory-1a054a382de0

3. place kaggle.json on drive: https://colab.research.google.com/gist/corrieann/7b1e460d3381bf058627bdaa64840122/kaggle-api.ipynb#scrollTo=o_EDbSbXpfng

4. how to dl data from kaggle: https://forums.fast.ai/t/how-to-download-data-for-lesson-2-from-kaggle-for-planet-competition/7684

5. Transfer between Google Drive and Colab VM:
https://gist.github.com/yt114/dc5d2fd4437f858bb73e38f0aba362c7


 #### Transfer between Google Drive and Colab VM (REMEMBER TO TURN ON THE GPU SETTING BEFORE THE SETUP BELOW)

In [0]:
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /content/colab_util_repo
!mv /content/colab_util_repo/colab_util.py colab_util.py 
!rm -r /content/colab_util_repo
!pip install pydrive

In [0]:
# From Colab VM to Drive (from /tmp in Colab VM)
from colab_util import *
drive_handler = GoogleDriveHandler()
import glob
archived_files = ['/content/fastai']

# Archived to /tmp folder
tar_file_path = create_archive('sample_archive', local_file_paths=archived_files, verbose=True)
tar_file_path

drive_handler.upload(tar_file_path, parent_path='test_folder/fastai')

In [0]:
# From Drive to Colab VM (to /tmp in Colab VM)
drive_handler.download('fastai_archive.tar.gz', 
                       target_path='test_folder/fastai/sample_archive.tar.gz')

In [0]:
!tar -xvzf community_images.tar.gz

In [0]:
# Create folder in Drive
test_folder_id = drive_handler.create_folder('test_folder')
test_folder_id
test_subfolder_id = drive_handler.create_folder('fastai', parent_path='test_folder')
test_subfolder_id

#### Install Pytorch, Fastai Packages

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision

!pip3 install spacy
!python -m spacy download en

!pip install Pillow==4.1.1
!pip install "fastai==0.7.0"
!pip install torchtext==0.2.3
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
!apt update && apt install -y libsm6 libxext6

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
!pip install image

%matplotlib inline
from fastai.imports import *

In [0]:
torch.cuda.is_available()

In [0]:
torch.backends.cudnn.enabled

In [0]:
mkdir fastai
!cd fastai
!git clone https://github.com/fastai/courses.git
!wget http://files.fast.ai/data/dogscats.zip && unzip -qq dogscats.zip -d data/
!mv data fastai
!wget http://files.fast.ai/part2/lesson14/rossmann.tgz && mkdir -p ~/data/rossmann
!tar -xzf rossmann.tgz -C ~/data/rossmann/
!rm rossmann.tgz
!rm dogscats.zip
!wget http://files.fast.ai/data/aclImdb.tgz
!tar -xvzf aclImdb.tgz -C data/
!rm aclImdb.tgz

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

#### Install Kaggle API and Data

In [0]:
!pip3 install kaggle
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

import json
token = {"username":"alexlwh","key":"94e2a05071ba6d3e646a5d64e9a8e228"}
with open("/content/.kaggle/kaggle.json", "w") as file:
    json.dump(token, file)

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/root/.kaggle/kaggle.json"  # NOTE: This is different from the Medium post!
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [0]:
!kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p ~/data/planet/
!kaggle competitions download -c planet-understanding-the-amazon-from-space -f test-jpg.tar.7z -p ~/data/planet/
!kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv.zip -p ~/data/planet/

In [0]:
!apt-get install p7zip-full
!p7zip -d ~/data/planet/test-jpg.tar.7z 
!p7zip -d ~/data/planet/train-jpg.tar.7z
!mv test-jpg.tar ~/data/planet
!mv train-jpg.tar ~/data/planet
!tar -xvf ~/data/planet/test-jpg.tar
!tar -xvf ~/data/planet/train-jpg.tar
!unzip ~/data/planet/train_v2.csv.zip -d ~/data/planet/

In [0]:
!ls ~/data/planet